In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd
from scipy.optimize import minimize
from capstone_library import *

# Hints
## Yield in a Chemical Reaction
This time you are trying to optimise another four-dimensional black-box. It corresponds to the yield of a chemical process after processing in some factory. This type of process tends to be unimodal. Try to find the combination of chemicals that maximizes the yield!

# Let's go!

Let's load the data.

In [2]:
X = np.load('initial_data/function_5/initial_inputs.npy')
y = np.load('initial_data/function_5/initial_outputs.npy')

In [3]:
# loading new data
new_queries = get_function_data_from_file('new_data/queries.txt', 5)
new_observ = get_function_data_from_file('new_data/observations.txt', 5)

In [4]:
# adding new_queries to X
new_queries = np.array(new_queries).reshape(-1, 4)
X = np.concatenate((X, new_queries), axis=0)

# adding new_observ to Y
new_observ = np.array(new_observ).reshape(-1)
y = np.concatenate((y, new_observ), axis=0)

## Visualizing the data and thinking of the problem

In [5]:
# visualising the data as a table
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))), columns=['x1', 'x2', 'x3', 'x4', 'y'])
df.head(100)

,x1,x2,x3,x4,y
0,0.191447,0.038193,0.607418,0.414584,64.443440
1,0.758653,0.536518,0.656000,0.360342,18.301380
2,0.438350,0.804340,0.210245,0.151295,0.112940
3,0.706051,0.534192,0.264243,0.482088,4.210898
4,0.836478,0.193610,0.663893,0.785649,258.370525
5,0.683432,0.118663,0.829046,0.567577,78.434389
6,0.553621,0.667350,0.323806,0.814870,57.571537
7,0.352356,0.322242,0.116979,0.473113,109.571876
8,0.153786,0.729382,0.422598,0.443074,8.847992
9,0.463442,0.630025,0.107906,0.957644,233.223610


In [6]:

# sort the data by the output, with the best value at the top
df = df.sort_values(by=['y'], ascending=False)
df.head(100)

,x1,x2,x3,x4,y
24,0.999999,0.999999,0.999999,0.999999,8662.405001
31,0.999999,0.999999,0.999999,0.999999,8662.405001
30,0.999999,0.999999,0.999999,0.999999,8662.405001
22,0.999999,0.999999,0.999999,0.999999,8662.405001
26,0.999999,0.999999,0.999999,0.999999,8662.405001
25,0.999999,0.999999,0.999999,0.999999,8662.405001
23,0.999999,0.999999,0.999999,0.999999,8662.405001
21,0.653323,0.999999,0.999999,0.999999,5129.255429
29,0.000000,0.999999,0.000000,0.999999,1616.625747
28,0.000000,0.999999,0.999999,0.000000,1616.625747


The first point above is promising. Let's use a gradient-based method, given that the function is known to be unimodal.

In [7]:
# Define the acquisition function to be optimized (negative UCB in this case)
def negative_acquisition(X_new, gpr, kappa):
    X_new = X_new.reshape(-1, len(X[0]))
    mean, std = gpr.predict(X_new, return_std=True)
    ucb = mean + kappa * std
    return -ucb  # we want to maximize UCB, so minimize negative UCB

def get_next_query(kappa, X, y):
    # Initialize and fit the gpr
    gpr = GaussianProcessRegressor()
    gpr.fit(X, y)

    # Define the bounds of the optimization problem, and a random initial point
    bounds = [(0, 0.999999), (0, 0.999999), (0, 0.999999), (0, 0.999999)]
    x0 = np.random.uniform(0, 1, size=4)  # random initialization

    # Perform the optimization using L-BFGS
    result = minimize(negative_acquisition, x0=x0, args=(gpr, kappa), bounds=bounds, method='L-BFGS-B')

    # The next query point is the one that maximizes the acquisition function
    next_query = result.x
    return next_query

In [8]:
# 
next_query = get_next_query(1, X, y)
print(format_query(next_query))

0.000000-0.999999-0.999999-0.000000


### After observation 23 (13th query)
Let's go with the query below: 

0.553621-0.667349-0.700000-0.567576